##### Implementing Backpropagation From Scratch on Python 3+
- Let's see if theory and practice are the same thing.

In [1]:
import numpy as np

In [ ]:
# Provide some structure of the network
class Network:
    def __init__(self, structure):
        self.structure = structure
        self.num_layers = len(structure)
        self.B_n = [np.random.randn(l, 1) for l in structure[1:]]
        self.W_n = [np.random.randn(l, next_l) for l, next_l in zip(structure[:-1], structure[1:])]